In [3]:
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt

from event_display import load_data, events_index_bounds

# Playing with awkward arrays

In [16]:
A = np.array([[1, 2], [3, 4], [5, 6]])

In [18]:
A[A>2]

array([3, 4, 5, 6])

In [8]:
B = ak.Array([[1, 2], [3, 4], [5, 6, 7]])

In [10]:
B > 2

<Array [[False, False], [...], [True, True, True]] type='3 * var * bool'>

In [12]:
C = ak.zeros_like(B)

In [23]:
C = ak.where(B < 2, B**2, C)

In [24]:
C

<Array [[1, 0], [9, 16], [25, 36, 49]] type='3 * var * int64'>

In [15]:
B[B>2]

<Array [[], [3, 4], [5, 6, 7]] type='3 * var * int64'>

# Loop vs Vectorialized

In [ ]:
def project2d_slow(x, y, z, detector_geom, experiment) : # project 3D PMT positions of an event to 2D unfolded cylinder (very ugly please Erwan do not look at this... I think it's Thomas' code anyway)

  cylinder_radius = detector_geom[experiment]['cylinder_radius']
  zMax = detector_geom[experiment]['height']/2
  zMin = -detector_geom[experiment]['height']/2

  N = len(x)
  xproj = []
  yproj = []

  if experiment == 'WCTE' : # WCTE bottom and top cap no symmetrical! top PMTs are further away from the last row of cylinder PMTs than the bottom PMTs, and beware of spherical structure of mPMTs
    
    # values adjusted by hand so as to correctly identify the top and bottom PMTs, maybe get info from WCSim in PMT id or something
    eps_top = 60
    eps_bottom = 50

  else :
    eps_top = 0.01
    eps_bottom = 0.01

  for i in range(N) :

    if z[i] < zMax - eps_top and z[i] > zMin + eps_bottom : # cylinder
      
      azimuth = np.arctan2(y[i], x[i])
      
      if azimuth < 0:
        azimuth = 2 * np.pi + azimuth

      xproj.append(cylinder_radius * (azimuth - np.pi))
      yproj.append(z[i])

    elif z[i] > zMax - eps_top : # top cap
      
      xproj.append(- y[i])
      yproj.append(x[i] + zMax + cylinder_radius)
   
    else : # bottom cap
      
      xproj.append(- y[i])
      yproj.append(- x[i] + zMin - cylinder_radius)

  return np.array(xproj),np.array(yproj)

def load_data_slow(path2events, events_file, detector_geom, experiment, events_to_display='all') : # load data from root file and project it to 2D

    print('Loading data...')
    file = up.open(path2events + events_file)
    events_root = file['root_event'] # TTree of events variables {'hitx', 'hity', 'hitz', 'charge', 'time'}

    n_events = len(events_root['hitx'].array()) # number of events

    hitx = events_root['hitx'].array(library='np')
    hity = events_root['hity'].array(library='np')
    hitz = events_root['hitz'].array(library='np')
    charge = events_root['charge'].array(library='np')
    time = events_root['time'].array(library='np')

    event_start, event_end = events_index_bounds(events_to_display, n_events)

    events_dic = {'xproj': [], 'yproj': [], 'charge': [], 'time': []} # python dictionary to store data

    print('2D projection...')

    for event_index in range(event_start, event_end) :

      if experiment == 'WCTE' : # WCTE is rotated in WCSim to have beam on the z axis, rotate it back to have cylinder axis on z axis like SK and HK, then rotate a tiny bit around z axis so as not to cut a column of PMTs in half (but also rotate the top and bottom caps though...)
      
        if event_index == 0 : print('Rotating WCTE events...')

        thetax = np.pi/2
        Rx = np.array([[1, 0, 0], [0, np.cos(thetax), -np.sin(thetax)], [0, np.sin(thetax), np.cos(thetax)]])
        thetaz = 4.53
        Rz = np.array([[np.cos(thetaz), -np.sin(thetaz), 0], [np.sin(thetaz), np.cos(thetaz), 0], [0, 0, 1]])

        hitR = Rz@Rx@np.array([hitx[event_index], hity[event_index], hitz[event_index]])
        hitx[event_index], hity[event_index], hitz[event_index] = hitR[0], hitR[1], hitR[2]

      x2D, y2D = project2d_slow(hitx[event_index], hity[event_index], hitz[event_index], detector_geom, experiment) 
      events_dic['xproj'].append(x2D)
      events_dic['yproj'].append(y2D)
      events_dic['charge'].append(charge[event_index])
      events_dic['time'].append(time[event_index])

    return events_dic, n_events

In [31]:
detector_geom = {'SK': {'height': 3620.0, 'cylinder_radius': 3368.15/2, 'PMT_radius': 25.4}, 'HK': {'height': 6575.1, 'cylinder_radius': 6480/2, 'PMT_radius': 25.4}, 'WCTE': {'height': 338.0, 'cylinder_radius': 369.6/2, 'PMT_radius': 4.0}}

path2event = '/sps/t2k/mferey/WCSim2ML/Data/WCTE/'
events_file = 'WCTE_Nu16cShort_SO_mu_200_1kMeV_10kevents.root'